In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import scipy.stats as st
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt

%load_ext line_profiler
%matplotlib inline

torch.set_default_tensor_type(torch.DoubleTensor)

device = torch.device('cuda:0') if torch.cuda.is_available else torch.device("cpu")
# device = torch.device("cpu")

In [ ]:
import os, sys 
from ssa_solvers.simulators import StochasticSimulator, DeterministicSimulator 
from ssa_solvers.data_class import SimulationData

end_time = 300
n_steps = 300
n_traj = 10000
time_grid_np = np.arange(0, end_time, int(end_time / n_steps))
time_grid_torch = torch.from_numpy(time_grid_np).to(device)

### Simulating in cis mRNA - sRNA  circuit

In [ ]:
from circuits.mrna_srna.mrna_srna_incis import mRNAsRNAInCis, cfg

data_set_incis = SimulationData(device=device)
reaction_system_incis = mRNAsRNAInCis(device=torch.device("cpu"))
ode_simulator = DeterministicSimulator(
    reaction_system=reaction_system_incis,
    cfg=cfg
)
reaction_system_incis = mRNAsRNAInCis(device=device)
ssa_simulator = StochasticSimulator(
    reaction_system=reaction_system_incis,
    data_set=data_set_incis,
    cfg=cfg,
    device=device
)
reaction_system_incis.params = {'beta_fmrna': 2}  # increasing fmRNA production rate
init_pops = torch.zeros((reaction_system_incis.n_species, ), dtype=torch.int64, device=device) #torch.randint(1, (reaction_system.n_species, ), device=device)
ssa_simulator.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)
ode_res_incis = ode_simulator.simulate(init_pops=np.zeros((reaction_system_incis.n_species,)), time_grid=time_grid_np)

### Simulating in trans mRNA - sRNA  circuit

In [ ]:
from circuits.mrna_srna.mrna_srna_intrans import mRNAsRNAInTrans, cfg

data_set_intrans = SimulationData(device=device)
reaction_system_intrans = mRNAsRNAInTrans(device=torch.device("cpu"))
ode_simulator = DeterministicSimulator(
    reaction_system=reaction_system_intrans,
    cfg=cfg
)
reaction_system_intrans = mRNAsRNAInTrans(device=device)
ssa_simulator = StochasticSimulator(
    reaction_system=reaction_system_intrans,
    data_set=data_set_intrans,
    cfg=cfg,
    device=device
)
init_pops = torch.zeros((reaction_system_intrans.n_species, ), dtype=torch.int64, device=device) #torch.randint(1, (reaction_system.n_species, ), device=device)
ssa_simulator.simulate(init_pops=init_pops, end_time=end_time, n_trajectories=n_traj)
ode_res_intrans = ode_simulator.simulate(init_pops=np.zeros((reaction_system_intrans.n_species,)), time_grid=time_grid_np)

### Comptuting the statistics on a grid

In [ ]:
means_incis = data_set_incis.mean(time_range=time_grid_torch)
stds_incis = data_set_incis.std(time_range=time_grid_torch)
cov_incis = data_set_incis.coefficient_of_variation(time_range=time_grid_torch)

means_intrans = data_set_intrans.mean(time_range=time_grid_torch)
stds_intrans = data_set_intrans.std(time_range=time_grid_torch)
cov_intrans = data_set_intrans.coefficient_of_variation(time_range=time_grid_torch)

### Plotting results

In [ ]:
species_idx_incis = 1
plt.figure()
plt.plot(time_grid_np, means_incis[species_idx_incis, :], 'b', label='Mean SSA (in cis)')
plt.fill_between(time_grid_np, means_incis[species_idx_incis,:]+stds_incis[species_idx_incis, :], means_incis[species_idx_incis,:]-stds_incis[species_idx_incis, :], 
                 color='b', alpha=0.3)
plt.xlim([0, end_time])    
plt.plot(time_grid_np, ode_res_incis[species_idx_incis, :], 'b--', label='ODE (in cis)')
plt.xlabel('Time (s)')
plt.ylabel('Number of species')
# random traces 
# trajs = [0, 40, 20, 11]
# for traj_idx in trajs:
#     plt.plot(data_set.raw_times_trajectories[traj_idx,  :], data_set.raw_pops_trajectories[traj_idx, species_idx, :], color='b')

species_idx_intrans = 2
plt.plot(time_grid_np, means_intrans[species_idx_intrans, :], 'r', label='Mean SSA (in trans)')
plt.fill_between(time_grid_np, means_intrans[species_idx_intrans,:]+stds_intrans[species_idx_intrans, :], means_intrans[species_idx_intrans,:]-stds_intrans[species_idx_intrans, :], 
                 color='r',alpha=0.3)
plt.xlim([0, end_time])    
plt.plot(time_grid_np, ode_res_intrans[species_idx_intrans, :], 'r--', label='ODE (in trans)')
plt.xlabel('Time (s)')
plt.ylabel('Number of species')
plt.xlim([0, max(time_grid_np)])
plt.ylim([0, 1.1 * max([(means_intrans[species_idx_intrans,:]+stds_intrans[species_idx_intrans, :]).max(), 
                        (means_incis[species_idx_incis,:]+stds_incis[species_idx_incis, :]).max()])])
plt.legend()
